In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
import keras
from keras import losses, optimizers, metrics

Using TensorFlow backend.


In [2]:
class ProteinLigandDataGenerator(keras.utils.Sequence):
    def __init__(self, csv_path, batch_size, shuffle=True, dim=(24,24,24), n_channels=2):
        self.df = self.prepare_df(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = None
        self.dim = dim
        self.n_channels = n_channels
        self.on_epoch_end()
    
    def prepare_df(self, csv_path):
        # oversampling happens here, 
        match_count = 1
        random_count = 4
        supplement = random_count - match_count
        
        df = pd.read_csv(csv_path)
        same_idx = df[df['pro_id'] == df['lig_id']]
        duplicates = [same_idx]*supplement
        return df.append(duplicates, ignore_index=True)
        
    
    def __len__(self):
        '''
        Batches per epoch
        '''
        return int(len(self.df.index)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.generate_data(indexes)
        
        return X, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df.index))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def generate_data(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        sub_df = self.df.iloc[indexes]
        sub_df.reset_index(inplace=True)
        for row in sub_df.itertuples():
            idx = row[0]
            full_X = np.load(row.dests)
            X[idx,:,:,:,:] = full_X
            y[idx] = row.score
            
        return X, y

In [3]:
train_generator = ProteinLigandDataGenerator('./data/csv/train_lig_pro_pairs.csv', 12)
test_generator = ProteinLigandDataGenerator('./data/csv/test_lig_pro_pairs.csv', 12)

In [7]:
print(len(train_generator))
print(400 * 7)

1801
2800


In [8]:
model = Sequential()
model.add(Conv3D(96, kernel_size=3, activation='relu', input_shape=(24,24,24,2)))
model.add(BatchNormalization())
model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool3D())
model.add(Conv3D(196, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv3D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling3D())
model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=1, decay=1e-5)
model.compile(loss=losses.mean_squared_error, optimizer=sgd, metrics=[metrics.mse])

In [9]:
history = model.fit_generator(train_generator, epochs=2, use_multiprocessing=True, workers=4, verbose=1)

Epoch 1/2
1801/1801 [==============================] - 304s 169ms/step - loss: 0.0504 - mean_squared_error: 0.0504
Epoch 2/2
1801/1801 [==============================] - 251s 139ms/step - loss: 0.0250 - mean_squared_error: 0.0250


In [12]:
test_history = model.evaluate_generator(test_generator, use_multiprocessing=True, workers=4)

In [13]:
print('train: ', history.history['mean_squared_error'])
print('test: ', test_history)

train:  [0.050404901412351426, 0.025024810155960888]
test:  [0.03852596025297659, 0.03852596025297659]


In [14]:
model.save('models/basic_c_and_n_regression_hydrophobic_only.h5')